In [1]:
import pandas as pd
import numpy as np
df=pd.read_csv("/Users/maloofbashir/Downloads/oasis_longitudinal.csv")

In [2]:
df['M/F']=df['M/F'].replace(['F','M'],[0,1])
df['Group']=df['Group'].replace(['Converted'],['Demented'])
df['Group']=df['Group'].replace(['Nondemented','Demented'],[0,1])
df=df.drop(['MRI ID' ,'Visit' , 'Hand'],axis=1)

mean_value=int(df['SES'].median())
df['SES'].fillna(value=mean_value,inplace=True)

mean_value_1=int(df['MMSE'].median())
df['MMSE'].fillna(value=mean_value,inplace=True)



In [3]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler 
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, roc_curve, auc

In [6]:
Y = df['Group'].values
X = df[['M/F', 'Age', 'EDUC', 'SES', 'MMSE', 'eTIV', 'nWBV', 'ASF']]
X_trainval, X_test, Y_trainval, Y_test = train_test_split(X, Y, random_state=0)
scaler = MinMaxScaler().fit(X_trainval)
X_trainval_scaled = scaler.transform(X_trainval)
X_test_scaled = scaler.transform(X_test)
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [9]:
acc=[]
tunning_range=[0.001, 0.1, 1, 10, 100]
kfolds=5
from sklearn.model_selection import KFold
best_score=0

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
acc=[]
tunning_range=[0.001, 0.1, 1, 10, 100]
kfolds=5
from sklearn.model_selection import KFold
best_score=0
 # set the number of folds
kfolds = KFold(n_splits=5, random_state=1, shuffle=True)

for c in [0.001, 0.1,0.5, 1 ,5, 10,20,50 ,100,200]:
    logRegModel = LogisticRegression(C=c)
    # perform cross-validation
    scores = cross_val_score(logRegModel, X_trainval, Y_trainval, cv=kfolds, scoring='accuracy') # Get recall for each parameter setting
    # compute mean cross-validation accuracy
    score = np.mean(scores)
    
    # Find the best parameters and score
    if score > best_score:
        vale_c=c
        best_score = score
        best_parameters = c

In [7]:
best_parameters=100
SelectedLogRegModel = LogisticRegression(C=best_parameters).fit(X_trainval_scaled, Y_trainval)

test_score = SelectedLogRegModel.score(X_test_scaled, Y_test)
PredictedOutput = SelectedLogRegModel.predict(X_test_scaled)
test_recall = recall_score(Y_test, PredictedOutput, pos_label=1)
fpr, tpr, thresholds = roc_curve(Y_test, PredictedOutput, pos_label=1)
test_auc = auc(fpr, tpr)
print("Test accuracy with best C parameter is", test_score)

Test accuracy with best C parameter is 0.8085106382978723


# SVM

In [10]:
best_score = 0

for c_paramter in [0.001, 0.01, 0.1, 1, 10, 100, 1000]: #iterate over the values we need to try for the parameter C
    for gamma_paramter in [0.001, 0.01, 0.1, 1, 10, 100, 1000]: #iterate over the values we need to try for the parameter gamma
        for k_parameter in ['rbf', 'linear', 'poly', 'sigmoid']: # iterate over the values we need to try for the kernel parameter
            svmModel = SVC(kernel=k_parameter, C=c_paramter, gamma=gamma_paramter) #define the model
            # perform cross-validation
            scores = cross_val_score(svmModel, X_trainval_scaled, Y_trainval, cv=kfolds, scoring='accuracy')
            # the training set will be split internally into training and cross validation

            # compute mean cross-validation accuracy
            score = np.mean(scores)
            # if we got a better score, store the score and parameters
            if score > best_score:
                best_score = score #store the score 
                best_parameter_c = c_paramter #store the parameter c
                best_parameter_gamma = gamma_paramter #store the parameter gamma
                best_parameter_k = k_parameter

In [11]:
SelectedSVMmodel = SVC(C=best_parameter_c, gamma=best_parameter_gamma, kernel=best_parameter_k,probability=True).fit(X_trainval_scaled, Y_trainval)

test_score = SelectedSVMmodel.score(X_test_scaled, Y_test)
PredictedOutput = SelectedSVMmodel.predict(X_test_scaled)
test_recall = recall_score(Y_test, PredictedOutput, pos_label=1)
fpr, tpr, thresholds = roc_curve(Y_test, PredictedOutput, pos_label=1)
test_auc = auc(fpr, tpr)

print("Test accuracy with the best parameters is", test_score)

Test accuracy with the best parameters is 0.8085106382978723


# Hybrid Model-1

### In this method SVM model is used to pre-process the data and then feed the output into a neural network for classification.

In [12]:
from sklearn.svm import SVC
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense


In [13]:
model = Sequential()

Metal device set to: Apple M1


2023-01-15 21:36:07.256479: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-01-15 21:36:07.256562: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [14]:
X_train_processed = SelectedSVMmodel.predict_proba(X_trainval_scaled)
X_test_processed = SelectedSVMmodel.predict_proba(X_test_scaled)

In [15]:
model.add(Dense(10, input_dim=X_train_processed.shape[1], activation='relu'))

In [16]:
model.add(Dense(1, activation='sigmoid'))

In [17]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [19]:
model.fit(X_train_processed,Y_trainval , epochs=10, batch_size=32)

Epoch 1/10


2023-01-15 21:37:20.613632: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-01-15 21:37:20.784080: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


9/9 [==============================] - 0s 8ms/step - loss: 0.6006 - accuracy: 1.0000
Epoch 2/10
9/9 [==============================] - 0s 5ms/step - loss: 0.5910 - accuracy: 1.0000
Epoch 3/10
9/9 [==============================] - 0s 5ms/step - loss: 0.5814 - accuracy: 1.0000
Epoch 4/10
9/9 [==============================] - 0s 5ms/step - loss: 0.5718 - accuracy: 1.0000
Epoch 5/10
9/9 [==============================] - 0s 5ms/step - loss: 0.5620 - accuracy: 1.0000
Epoch 6/10
9/9 [==============================] - 0s 4ms/step - loss: 0.5521 - accuracy: 1.0000
Epoch 7/10
9/9 [==============================] - 0s 5ms/step - loss: 0.5420 - accuracy: 1.0000
Epoch 8/10
9/9 [==============================] - 0s 4ms/step - loss: 0.5317 - accuracy: 1.0000
Epoch 9/10
9/9 [==============================] - 0s 5ms/step - loss: 0.5213 - accuracy: 1.0000
Epoch 10/10
9/9 [==============================] - 0s 5ms/step - loss: 0.5107 - accuracy: 1.0000


In [21]:
test_loss, test_acc = model.evaluate(X_test_processed, Y_test)
print("Test accuracy: {:.2f}%".format(test_acc*100))

3/3 [==============================] - 0s 12ms/step - loss: 0.5956 - accuracy: 0.8298
Test accuracy: 82.98%


2023-01-15 21:37:38.005284: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


## So with this model we are able to achieve accuracy of 82.98%

# HYBRID MODEL-II

### One approach is to use the SVM model to select the most important features of the data, and then use the selected features as input to the neural network.

In [22]:
from sklearn.feature_selection import SelectFromModel

In [25]:
svm = SVC(kernel='linear', probability=True, C=0.1)
svm.fit(X_trainval_scaled, Y_trainval)

SVC(C=0.1, kernel='linear', probability=True)

In [26]:
selector = SelectFromModel(svm, prefit=True)
X_train_processed = selector.transform(X_trainval_scaled)
X_test_processed = selector.transform(X_test_scaled)

In [27]:
model = Sequential()

In [28]:
model.add(Dense(10, input_dim=X_train_processed.shape[1], activation='relu'))

In [29]:
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [30]:
model.fit(X_train_processed, Y_trainval, epochs=10, batch_size=32)

Epoch 1/10
9/9 [==============================] - 0s 8ms/step - loss: 0.6513 - accuracy: 0.6165
Epoch 2/10
9/9 [==============================] - 0s 5ms/step - loss: 0.6494 - accuracy: 0.6165
Epoch 3/10
1/9 [==>...........................] - ETA: 0s - loss: 0.6473 - accuracy: 0.6250

2023-01-15 21:43:12.204103: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


9/9 [==============================] - 0s 5ms/step - loss: 0.6482 - accuracy: 0.6165
Epoch 4/10
9/9 [==============================] - 0s 5ms/step - loss: 0.6467 - accuracy: 0.6165
Epoch 5/10
9/9 [==============================] - 0s 5ms/step - loss: 0.6456 - accuracy: 0.6165
Epoch 6/10
9/9 [==============================] - 0s 5ms/step - loss: 0.6442 - accuracy: 0.6165
Epoch 7/10
9/9 [==============================] - 0s 5ms/step - loss: 0.6426 - accuracy: 0.6165
Epoch 8/10
9/9 [==============================] - 0s 4ms/step - loss: 0.6411 - accuracy: 0.6165
Epoch 9/10
9/9 [==============================] - 0s 5ms/step - loss: 0.6399 - accuracy: 0.6165
Epoch 10/10
9/9 [==============================] - 0s 5ms/step - loss: 0.6388 - accuracy: 0.6165


In [31]:
test_loss, test_acc = model.evaluate(X_test_processed, Y_test)
print("Test accuracy: {:.2f}%".format(test_acc*100))

3/3 [==============================] - 0s 11ms/step - loss: 0.6656 - accuracy: 0.5957
Test accuracy: 59.57%


2023-01-15 21:43:15.878233: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


## Getting the model accurcay of 59.57%

# HYBRID MODEL-III

### ### use the SVM model to select the most important features of the data, and then use the selected features as input to the neural network.

In [32]:
from sklearn.svm import SVC
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import concatenate
from keras.layers import Input
from keras.models import Model

In [33]:

svm_train_pred = SelectedSVMmodel.predict_proba(X_trainval_scaled)
svm_test_pred = SelectedSVMmodel.predict_proba(X_test_scaled)

In [34]:
input_nn = Input(shape=(X_trainval_scaled.shape[1],))
input_svm = Input(shape=(svm_train_pred.shape[1],))

In [35]:
concat = concatenate([input_nn, input_svm])

In [36]:
hidden = Dense(10, activation='relu')(concat)
output = Dense(1, activation='sigmoid')(hidden)

In [37]:
model = Model(inputs=[input_nn, input_svm], outputs=output)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [38]:
model.fit([X_trainval_scaled, svm_train_pred], Y_trainval, epochs=10, batch_size=32)

Epoch 1/10
9/9 [==============================] - 0s 9ms/step - loss: 0.7934 - accuracy: 0.3692
Epoch 2/10
1/9 [==>...........................] - ETA: 0s - loss: 0.8271 - accuracy: 0.2812

2023-01-15 21:43:47.869500: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


9/9 [==============================] - 0s 5ms/step - loss: 0.7608 - accuracy: 0.3369
Epoch 3/10
9/9 [==============================] - 0s 7ms/step - loss: 0.7309 - accuracy: 0.3082
Epoch 4/10
9/9 [==============================] - 0s 5ms/step - loss: 0.7037 - accuracy: 0.3441
Epoch 5/10
9/9 [==============================] - 0s 5ms/step - loss: 0.6784 - accuracy: 0.5520
Epoch 6/10
9/9 [==============================] - 0s 5ms/step - loss: 0.6540 - accuracy: 0.6165
Epoch 7/10
9/9 [==============================] - 0s 5ms/step - loss: 0.6314 - accuracy: 0.6380
Epoch 8/10
9/9 [==============================] - 0s 5ms/step - loss: 0.6090 - accuracy: 0.6452
Epoch 9/10
9/9 [==============================] - 0s 6ms/step - loss: 0.5880 - accuracy: 0.6487
Epoch 10/10
9/9 [==============================] - 0s 5ms/step - loss: 0.5677 - accuracy: 0.6559


In [39]:
test_loss, test_acc = model.evaluate([X_test_scaled, svm_test_pred], Y_test)
print("Test accuracy: {:.2f}%".format(test_acc*100))

3/3 [==============================] - 0s 14ms/step - loss: 0.6212 - accuracy: 0.6064
Test accuracy: 60.64%


2023-01-15 21:43:50.885160: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


# Getting the model accuracy of 60.64%